In [1]:
import os
import re

from pyqir import BasicQisBuilder, Module, SimpleModule
from qir_trans import load

In [2]:
def save_pyqir_module(module: Module, name: str, output_dir: str = "QIR-example") -> None:
    """Save a PyQIR module to both .ll and .bc files in the specified directory.

    Args:
        module (Module): A PyQIR Module object (usually from `pyqir.SimpleModule`)
        output_dir (str): Directory to store output files (default is "QIR-example")
    """
    # Ensure directory exists
    os.makedirs(output_dir, exist_ok=True)

    # Determine file paths
    ll_path = os.path.join(output_dir, f"{name}.ll")
    bc_path = os.path.join(output_dir, f"{name}.bc")

    # Write .ll
    with open(ll_path, "w") as f:
        if type(module) is SimpleModule:
            f.write(str(module.ir()))  # str returns IR text
        else:
            f.write(str(module))

    # Write .bc (bitcode)
    with open(bc_path, "wb") as f:
        if type(module) is SimpleModule:
            f.write(module.bitcode())
        else:
            f.write(module.bitcode)

    print(f"Saved QIR module '{name}' to:\n  - {ll_path}\n  - {bc_path}")

In [3]:
# Create the module with two qubits and two results.
bell = SimpleModule("bell", num_qubits=2, num_results=2)
qis = BasicQisBuilder(bell.builder)

# Add instructions to the module to create a Bell pair and measure both qubits.
qis.cx(bell.qubits[0], bell.qubits[1])
qis.cz(bell.qubits[0], bell.qubits[1])
qis.rz(10, bell.qubits[0])
qis.h(bell.qubits[0])
qis.mz(bell.qubits[0], bell.results[0])
qis.mz(bell.qubits[1], bell.results[1])
save_pyqir_module(bell, "bell_pair")

Saved QIR module 'bell_pair' to:
  - QIR-example/bell_pair.ll
  - QIR-example/bell_pair.bc


In [4]:
import pyqir
from pyqir import (
    BasicBlock,
    Builder,
    Context,
    Function,
    Linkage,
)

context = Context()
mod = pyqir.qir_module(
    context,
    "dynamic_allocation",
    qir_major_version=1,
    qir_minor_version=0,
    dynamic_qubit_management=True,
    dynamic_result_management=True,
)
builder = Builder(context)

# define external calls and type definitions
qubit_type = pyqir.qubit_type(context)
result_type = pyqir.result_type(context)

# PyQIR assumes you want to use static allocation for qubits and results, but
# you can still use dynamic allocation by manually calling the appropriate
# runtime functions.
qubit_allocate = Function(
    pyqir.FunctionType(qubit_type, []),
    Linkage.EXTERNAL,
    "__quantum__rt__qubit_allocate",
    mod,
)

qubit_release = Function(
    pyqir.FunctionType(pyqir.Type.void(context), [qubit_type]),
    Linkage.EXTERNAL,
    "__quantum__rt__qubit_release",
    mod,
)

result_get_one = Function(
    pyqir.FunctionType(result_type, []),
    Linkage.EXTERNAL,
    "__quantum__rt__result_get_one",
    mod,
)

result_equal = Function(
    pyqir.FunctionType(pyqir.IntType(context, 1), [result_type, result_type]),
    Linkage.EXTERNAL,
    "__quantum__rt__result_equal",
    mod,
)

m = Function(
    pyqir.FunctionType(result_type, [qubit_type]),
    Linkage.EXTERNAL,
    "__quantum__qis__m__body",
    mod,
)

# Create entry point
num_qubits = 1
num_results = 1
entry_point = pyqir.entry_point(mod, "main", num_qubits, num_results)
builder.insert_at_end(BasicBlock(context, "entry", entry_point))

# Define entry point body
qubit_return = builder.call(qubit_allocate, [])

assert qubit_return is not None
qubit = qubit_return

qis = pyqir.BasicQisBuilder(builder)
qis.h(qubit)

# Instead of qis.mz, use __quantum__qis__m__body.
result = builder.call(m, [qubit])
assert result is not None

# Instead of if_result, use __quantum__rt__result_equal and mod.if_.
one = builder.call(result_get_one, [])
assert one is not None
result_is_one = builder.call(result_equal, [result, one])
assert result_is_one is not None
builder.if_(result_is_one, lambda: qis.reset(qubit))

# Be sure to release any allocated qubits when you're done with them.
builder.call(qubit_release, [qubit])

# Add the termination of the entry point function.
_ = builder.ret(None)

# Verify the module and print it
error = mod.verify()
if error is not None:
    raise ValueError(error)

save_pyqir_module(mod, "dynamic_allocation")

Saved QIR module 'dynamic_allocation' to:
  - QIR-example/dynamic_allocation.ll
  - QIR-example/dynamic_allocation.bc


In [5]:
# path = "QIR-example/dynamic_allocation.ll"
# mod = load(path)

# # Check Declaration
# print("---Checking Function Declaration---\n")
# for func in mod.functions:
#     if func.is_declaration:
#         print(f"Function: {func.name}")
#         func_type = func.type
#         print("func_type:", func_type)
#         # print("func_type.element_type:", func_type.element_type)

#         for k, element in enumerate(func_type.element_type.elements):
#             if k == 0:
#                 print(
#                     "Return Type:\n\t", element,
#                     "\tVOID!!" if element.type_kind == 0 else None
#                 )
#             else:
#                 print(f"Params {k} Type:\n\t", element)
#                 # print("\t ", "element.is_opaque_struct:", element.is_opaque_struct)
#                 # if element.is_pointer:
#                 #     print("\t\t", "element.element_type", element.element_type)
#             print()
#         print()


# for func in mod.functions:
#     # check main function definition
#     if not func.is_declaration:
#         main = func

Test pyqir

In [6]:
from pyqir import qis

In [7]:
# TEST CASE FILE
# path = "QIR-example/bernstein_vazirani.ll"
# path = "QIR-example/test.ll"
path = "QIR-example/bell_pair.ll"
# path = "QIR-example/dynamic_allocation.bc"

# Need to implement
# path = "QIR-example/dynamic_allocation.bc"
# path = "QIR-example/CUDAQ_bell.ll"
# path = "QIR-example/base_profile.ll"

In [8]:
mod = load(path)

In [9]:
print("---Checking Global Variables---\n")
for k, var in enumerate(mod.global_variables):
    print(f"Variables: \t {var}")
    print(f"\t var.type: \t {var.type}")
    print(f"\t var.name: \t {var.name}")
    print(f"\t var.value: \t {var.get_constant_value()}")
    print()

---Checking Global Variables---



In [10]:
print("---Checking Global Structure---\n")
for k, s_type in enumerate(mod.struct_types):
    print(f"Structure: \t {s_type}")
    print(f"\t name: \t {s_type.name}")
    print(f"\t is_opaque: \t {s_type.is_opaque_struct}")
    print()

---Checking Global Structure---

Structure: 	 %Qubit = type opaque
	 name: 	 Qubit
	 is_opaque: 	 True

Structure: 	 %Result = type opaque
	 name: 	 Result
	 is_opaque: 	 True



In [11]:
func_table = {}

# Check Declaration
print("---Checking Function Declaration---\n")
for func in mod.functions:
    if func.is_declaration:
        func_table[func.name] = func.type

        print(f"Declaration: \t {func}", end="")
        print(f"Function Name: \t {func.name}")
        func_type = func.type
        print(f"Function Type: \t {func_type.element_type}")
        # print("func_type.element_type:", func_type.element_type)
        for k, element in enumerate(func_type.element_type.elements):
            if k == 0:
                print(
                    "Return Type: \t",
                    element,
                    # "\t---VOID!!!" if element.type_kind == 0 else ''
                )
            else:
                print(f"Params [{k - 1}] Type:\n\t", element)
                # print("\t ", "element.is_opaque_struct:", element.is_opaque_struct)
                if element.is_pointer:
                    print(
                        "\t\t",
                        element.element_type.name,
                        "is_struct",
                        element.element_type.is_struct,
                    )
        print("\n")

---Checking Function Declaration---

Declaration: 	 declare void @__quantum__qis__cnot__body(%Qubit*, %Qubit*)
Function Name: 	 __quantum__qis__cnot__body
Function Type: 	 void (%Qubit*, %Qubit*)
Return Type: 	 void
Params [0] Type:
	 %Qubit*
		 Qubit is_struct True
Params [1] Type:
	 %Qubit*
		 Qubit is_struct True


Declaration: 	 declare void @__quantum__qis__cz__body(%Qubit*, %Qubit*)
Function Name: 	 __quantum__qis__cz__body
Function Type: 	 void (%Qubit*, %Qubit*)
Return Type: 	 void
Params [0] Type:
	 %Qubit*
		 Qubit is_struct True
Params [1] Type:
	 %Qubit*
		 Qubit is_struct True


Declaration: 	 declare void @__quantum__qis__rz__body(double, %Qubit*)
Function Name: 	 __quantum__qis__rz__body
Function Type: 	 void (double, %Qubit*)
Return Type: 	 void
Params [0] Type:
	 double
Params [1] Type:
	 %Qubit*
		 Qubit is_struct True


Declaration: 	 declare void @__quantum__qis__h__body(%Qubit*)
Function Name: 	 __quantum__qis__h__body
Function Type: 	 void (%Qubit*)
Return Type: 	

In [12]:
func_table

{'__quantum__qis__cnot__body': <llvmlite.binding.typeref.TypeRef at 0x1147eda10>,
 '__quantum__qis__cz__body': <llvmlite.binding.typeref.TypeRef at 0x1147f49d0>,
 '__quantum__qis__rz__body': <llvmlite.binding.typeref.TypeRef at 0x11464d890>,
 '__quantum__qis__h__body': <llvmlite.binding.typeref.TypeRef at 0x1147f5e50>,
 '__quantum__qis__mz__body': <llvmlite.binding.typeref.TypeRef at 0x11462d390>}

In [13]:
call_table = []
var_table = []
op_table = []

In [14]:
for func in mod.functions:
    # check main function definition
    attr_dict = {}
    for attr in func.attributes:
        matches = re.findall(r'"(\w+)"(?:="([^"]*)")?', attr.decode())
        for k, v in matches:
            if v != "":
                attr_dict[k] = v
            else:
                attr_dict[k] = True

    if "entry_point" in attr_dict.keys():
        # if not func.is_declaration:
        main = func
        for block in main.blocks:
            print(f"Block: \t {block.name}")
            for inst in block.instructions:
                print(f"Instuction: \t {inst}")
                print(f"\t Opcode: \t {inst.opcode}")
                # print(f"\t Visibility: \t {inst.visibility}")
                print(f"\t Return Type: \t {inst.type}")

                if inst.type.type_kind != "void":
                    var_table.append(inst)

                l = len([1 for _ in inst.operands])

                if inst.opcode == "call":
                    for k, op in enumerate(inst.operands):
                        if k == l - 1:
                            print(f"\t Function Name:\t {op}")

                            call_table.append((op.name, op.type))

                        else:
                            # operands[-1] is the name of called function
                            print(f"\t Operands[{k}]: \t {op}")
                            print(f"\t\t op.type: \t {op.type}")
                            if op.is_constant:
                                print(f"\t\t op.value: \t {op.get_constant_value()}")
                            else:
                                print(f"\t\t op.value_kind:\t {op.value_kind.name}")
                                print(f"\t\t op.parents: \t {op._parents['instruction']}")
                                op_table.append(op)

                            # print(op)
                            # print(f"\t\t op.value: \t {op.get_constant_value()}")
                            # print(f"\t\t\t op.type.element_type: {op.type.element_type}")
                            # print(f"\t\t\t op.name: {op.type.element_type.name}")
                            # op_types.append(op.type.element_type)

                elif inst.opcode == "br":
                    if l == 1:
                        for k, op in enumerate(inst.operands):
                            print(f"\t Operands[{k}]: \t label %{op.name}")
                            print(f"\t\t op.type: \t {op.type}")

                    elif l == 3:
                        for k, op in enumerate(inst.operands):
                            if k == 0:
                                print(f"\t Operands[{k}]: \t {op}")
                                print(f"\t\t op.type: \t {op.type}")
                                if op.is_constant:
                                    print(f"\t\t op.value: \t {op.get_constant_value()}")
                                else:
                                    print(f"\t\t op.value_kind:\t {op.value_kind.name}")
                            else:
                                print(f"\t Operands[{k}]: \t label %{op.name}")
                                print(f"\t\t op.type: \t {op.type}")
                print()
            print("\n")

Block: 	 entry
Instuction: 	   call void @__quantum__qis__cnot__body(%Qubit* null, %Qubit* inttoptr (i64 1 to %Qubit*))
	 Opcode: 	 call
	 Return Type: 	 void
	 Operands[0]: 	 %Qubit* null
		 op.type: 	 %Qubit*
		 op.value: 	 %Qubit* null
	 Operands[1]: 	 %Qubit* inttoptr (i64 1 to %Qubit*)
		 op.type: 	 %Qubit*
		 op.value: 	 %Qubit* inttoptr (i64 1 to %Qubit*)
	 Function Name:	 declare void @__quantum__qis__cnot__body(%Qubit*, %Qubit*)


Instuction: 	   call void @__quantum__qis__cz__body(%Qubit* null, %Qubit* inttoptr (i64 1 to %Qubit*))
	 Opcode: 	 call
	 Return Type: 	 void
	 Operands[0]: 	 %Qubit* null
		 op.type: 	 %Qubit*
		 op.value: 	 %Qubit* null
	 Operands[1]: 	 %Qubit* inttoptr (i64 1 to %Qubit*)
		 op.type: 	 %Qubit*
		 op.value: 	 %Qubit* inttoptr (i64 1 to %Qubit*)
	 Function Name:	 declare void @__quantum__qis__cz__body(%Qubit*, %Qubit*)


Instuction: 	   call void @__quantum__qis__rz__body(double 1.000000e+01, %Qubit* null)
	 Opcode: 	 call
	 Return Type: 	 void
	 Ope

In [15]:
op_table

[]

In [16]:
for op in op_table:
    print(op in var_table)

In [17]:
# for name, func_type in call_table:
#     print(func_type, func_table[name])
#     print(func_type == func_table[name])

In [18]:
import sys

sys.path.insert(0, os.path.abspath("."))
from qir_trans.translator import Exporter

print(Exporter().dumps(mod))

OPENQASM 3.0;
include "stdgates.inc";
qubit[2] Qubit;
bit[2] Result;
cx Qubit[0], Qubit[1];
cz Qubit[0], Qubit[1];
rz(10.0) Qubit[0];
h Qubit[0];
Result[0] = measure Qubit[0];
Result[1] = measure Qubit[1];



In [19]:
# # Get the list of QIS calls
# calls: List[Call] = []

# for block in entry_point.basic_blocks:
#     for inst in block.instructions:
#         if isinstance(inst, Call):
#             calls.append(inst)

In [20]:
# c = calls[1]
# c.args[0]

In [21]:
# from qiskit.utils import optionals as _optionals
# _optionals